<br/><font size=10>Feature extraction</font><br/>

Feature extraction 是指通過領域知識從輸入信號中提取區分性特徵的過程。傳統特徵從 time-domain（例如，variance、mean value、kurtosis）、frequency-domain（例如，fast Fourier transform）和 time-frequency domains（例如，discrete wavelet transform）中提取。它們將豐富有關用戶意圖的可區分信息。<sup>1</sup>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Manual-feature-extraction" data-toc-modified-id="Manual-feature-extraction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Manual feature extraction</a></span></li><li><span><a href="#Automatical-feature-extraction" data-toc-modified-id="Automatical-feature-extraction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Automatical feature extraction</a></span><ul class="toc-item"><li><span><a href="#Autoencoder" data-toc-modified-id="Autoencoder-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Autoencoder</a></span></li></ul></li><li><span><a href="#Reference" data-toc-modified-id="Reference-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# Manual feature extraction 

手動 feature extraction 高度依賴於領域知識。例如，從 motor imagery EEG signal 提取獨特特徵需要神經科學知識。手動特徵提取也非常耗時且困難。當從大腦信號中手動提取特徵時，一些區分性特徵如 time-frequency features、wavelet entropy 和 band-specific power 是常見的選擇。

深度學習的一個優勢是它可以在沒有領域知識的情況下自動學習 informative features 並發現 underlying patterns 。<font color="red">在本教程中，我們只介紹如何從原始數據中學習 representative features，而不討論傳統的特徵提取方法。</font>

# Automatical feature extraction

Representative deep learning models 可以從輸入數據中自動學習純粹的 representative features
這些 algorithms 僅具有特徵提取的功能，<font color="red">但不能 classification</font>

> 如下圖所示<sup>1</sup>：  
代表性模型可以分為 Autoencoder (AE)、Restricted Boltzmann Machine (RBM) 和 Deep Belief Networks (DBN)。D-AE 表示 DeepAutoencoder，指的是具有多個隱藏層的 Autoencoder。同樣，D-RBM 表示 Deep-Restricted Boltzmann Machine，具有多個隱藏層。Deep Belief Network 可以由 AE 或 RBM 組成，因此，我們將 DBN 分為 DBN-AE 和 DBN-RBM。

![avatar](https://raw.githubusercontent.com/xiangzhang1015/ML_BCI_tutorial/main/tutorial/dlm.PNG)

__常用的深度學習表示算法有 AE、RBM、DBN 及其變體。__

## Autoencoder
In this part, as an example of unsupervised feature extraction via deep learning, we will present the implementation of a simple framework that use AE as a feature extractor and feed the learned features to a standard KNN classifier.

AutoEncoder：用於從原始數據中提取有用的特徵。這些特徵可以捕捉數據的潛在結構，並且可以用於後續的分類任務。

KNN：用於評估 AutoEncoder 提取的特徵的分類效果。KNN 是一種簡單且直觀的分類算法，適合用來快速評估特徵的質量。

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.utils.data as Data
import myimporter
from BCI_functions import *  # BCI_functions.ipynb contains some functions we might use multiple times in this tutorial

dataset_1 = np.load('1.npy')
print('dataset_1 shape:', dataset_1.shape)

# 移除 label 
removed_label = [2, 3, 4 ] # 至少 10 一定要移除（resting state）
dataset_1 = dataset_1[~np.isin(dataset_1[:, -1], removed_label)]

# data segmentation
n_class = int(11-len(removed_label))  # 0~9 classes ('10:rest' is not considered)
no_feature = 64  # the number of the features
segment_length = 8  # selected time window; 16=160*0.1
LR = 0.005  # learning rate
EPOCH = 401

data_seg = extract(dataset_1, n_classes=n_class, n_fea=no_feature, time_window=segment_length, moving=(segment_length/2))  # 50% overlapping
print('After segmentation, the shape of the data:', data_seg.shape)

# split training and test data
no_longfeature = no_feature*segment_length
data_seg_feature = data_seg[:, :no_longfeature]
data_seg_label = data_seg[:, no_longfeature:no_longfeature+1]
train_feature, test_feature, train_label, test_label = train_test_split(data_seg_feature, data_seg_label, shuffle=True)

# normalization
# before normalize reshape data back to raw data shape
train_feature_2d = train_feature.reshape([-1, no_feature])
test_feature_2d = test_feature.reshape([-1, no_feature])

# min-max normalization
from sklearn.preprocessing import MinMaxScaler
scaler3 = MinMaxScaler().fit(train_feature)
train_fea_norm1 = scaler3.transform(train_feature)
test_fea_norm1 = scaler3.transform(test_feature)
print('After normalization, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter normalization, the shape of test feature:', test_fea_norm1.shape)

# 將 noramalize 後的數據重新塑形為 3D，以便輸入到 AutoEncoder  中
train_fea_norm1 = train_fea_norm1.reshape([-1, segment_length, no_feature])
test_fea_norm1 = test_fea_norm1.reshape([-1, segment_length, no_feature])
print('After reshape, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter reshape, the shape of test feature:', test_fea_norm1.shape)

BATCH_size = train_fea_norm1.shape[0] # use test_data as batch size

# feed data into dataloader
train_fea_norm1 = torch.tensor(train_fea_norm1).type('torch.FloatTensor')
train_label = torch.tensor(train_label.flatten())
train_data = Data.TensorDataset(train_fea_norm1, train_label)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_size, shuffle=False)

test_fea_norm1 = torch.tensor(test_fea_norm1).type('torch.FloatTensor')
test_label = torch.tensor(test_label.flatten())

class AutoEncoder(nn.Module): # encoder 將輸入數據壓縮到 64*4 維，decoder 將其重建回原始維度
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(no_feature*segment_length, 64*4),
        )
        self.decoder = nn.Sequential(
            nn.Linear(64*4, no_feature*segment_length),
            nn.Sigmoid(),
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

autoencoder = AutoEncoder()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR)
loss_func = nn.MSELoss()

best_acc = []

# classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)

# training and testing
for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):

        train_x = train_x.view(-1, no_feature*segment_length)
        train_encoded, train_decoded = autoencoder(train_x)

        loss = loss_func(train_decoded, train_x)  # mean square error
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients

        if epoch % 50 == 0 :
            knn.fit(train_encoded.data.numpy(), train_y.data.numpy())
            test_fea_norm1 = test_fea_norm1.view(-1, no_feature*segment_length)
            test_encoded, test_decoded = autoencoder(test_fea_norm1)
            knn_acc = knn.score(test_encoded.data.numpy(), test_label.data.numpy())

            print('Epoch: ', epoch, '| STEP: ', step, '|Autoencoder train loss: %.4f' % loss.item(),'|KNN accuracy: %.4f' % knn_acc)
            best_acc.append(knn_acc)

print('BEST TEST ACC: {}'.format(max(best_acc)))

dataset_1 shape: (259520, 65)
After segmentation, the shape of the data: (53628, 513)
After normalization, the shape of training feature: (40221, 512) 
After normalization, the shape of test feature: (13407, 512)
After reshape, the shape of training feature: (40221, 8, 64) 
After reshape, the shape of test feature: (13407, 8, 64)
Epoch:  0 | STEP:  0 |Autoencoder train loss: 0.0084 |KNN accuracy: 0.6893
Epoch:  50 | STEP:  0 |Autoencoder train loss: 0.0029 |KNN accuracy: 0.6611
Epoch:  100 | STEP:  0 |Autoencoder train loss: 0.0023 |KNN accuracy: 0.6329
Epoch:  150 | STEP:  0 |Autoencoder train loss: 0.0016 |KNN accuracy: 0.6472
Epoch:  200 | STEP:  0 |Autoencoder train loss: 0.0015 |KNN accuracy: 0.6506
Epoch:  250 | STEP:  0 |Autoencoder train loss: 0.0018 |KNN accuracy: 0.6482
Epoch:  300 | STEP:  0 |Autoencoder train loss: 0.0202 |KNN accuracy: 0.4378
Epoch:  350 | STEP:  0 |Autoencoder train loss: 0.0015 |KNN accuracy: 0.6506
Epoch:  400 | STEP:  0 |Autoencoder train loss: 0.0010 

_如上所見，這種組合的實驗表現不佳。原因可能追溯到數據集的特性。然而，對於不同的數據集，例如具有 fMRI 數據的數據集，autoencoder 可能會取得更好的表現。_

# Reference

<div id="refer-anchor-1"></div>

- [1]  [Zhang, X., Yao, L., Wang, X., Monaghan, J.J., Mcalpine, D. and Zhang, Y., 2020. A survey on deep learning-based non-invasive brain signals: recent advances and new frontiers. Journal of Neural Engineering.](https://iopscience.iop.org/article/10.1088/1741-2552/abc902/meta)